In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'NanumGothicCoding'
plt.rcParams['font.size'] = 10
import seaborn as sns

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error  
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import precision_score 
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

***
선형 회귀 분석은 예측 문제를 풀기에는 적합하지만 분류 문제를 풀기에는 적합하지 않다.
***

선형 회귀 분석의 레이블 데이터 값의 범위에는 제한이 없기 때문에 결과값이 제한되는 상황에서 회귀 모형이 결과 값에 제한이 없다면 분류 문제를 풀기 어려워진다. 0과 1로 분류해야 하는 문제에서 모델의 결과값는 오직 0과 1 사이의 값으로 나와야 할 것이다. 이러한 문제점을 해결하기 위해 사용하는 방법이 로지스틱 회귀 분석이다.

$$z = w^Tx + b$$

위 식은 선형 회귀 분석 모델이다.  
z값은 제한이 없고 어떤 값도 가질 수 있으므로 위 식을 이용해서 분류 문제를 푸는 것은 어렵다는 것을 알 수 있다. 기존의 선형 회귀 모델 식을 분류 문제를 풀 수 있도록 변형시키는 과정이 필요하다. 이를 위해 결과값이 제한된 범위를 가지도록 변형시켜 보자.

$$y = \frac{1}{1 + e^{-z}} = \frac{1}{1 + e^{-(w^Tx + b)}}$$

z에 대한 선형 회귀 식을 위와 같이 변경시키면 새로운 출력 y는 0과 1 사이의 값만 가지게 된다. 위 함수를 시그모이드(sigmoid) 함수라고 부른다. 시그모이드 함수는 딥러닝에서도 자주 나오는 함수로 식의 우변이 $z = w^Tx + b$ 형태가 되도록 변형시키면 아래와 같이 표현할 수 있다.

$$log(\frac{y}{1 - y}) = w^Tx + b$$

위식을 보면 우변이 $z = w^Tx + b$와 같은 선형 형태로 나타난다. 위 식에서 좌변 $\frac{y}{1 - y}$를 오즈 비(odds ratio)라고 부른다.

오즈 비에서 분자에 해당하는 y가 사건이 발생할 확률(성공 확률)이라고 했을 때 분모인 1 - y는 사건이 발생하지 않을 확률(실패 확률)에 해당된다. 실패 확률과 성공 확률의 비를 오즈 비라고 부른다. 또한 오즈 비에 log를 최한 값 $log(\frac{y}{1 - y})$을 로짓(logit)이라고 부른다.

***
위스콘신 대학 암 발생 데이터를 사용해 양성과 악성을 예측하는 모델을 생성하고 학습시킨다.
***

In [2]:
# 데이터 불러오기
raw_data = datasets.load_breast_cancer() # 사이킷런이 제공하는 위스콘신 암 발생 데이터를 불러온다.

# 피쳐, 레이블 데이터 저장
xData = raw_data.data # 피쳐 데이터를 저장한다.
yData = raw_data.target # 피쳐 데이터에 따른 레이블을 저장한다.
print(xData.shape, yData.shape)


# 학습 데이터와 테스트 데이터로 분할
x_train, x_test, y_train, y_test = train_test_split(xData, yData, random_state=1)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

# 데이터 표준화
std_scale = StandardScaler() # 표준화 스케일러 객체를 만든다.
x_train = std_scale.fit_transform(x_train) # 학습 데이터를 스케일러로 표준화하고 적용한다.
x_test = std_scale.transform(x_test) # 테스트 데이터를 학습 데이터로 표준화된 스케일러에 적용한다.

#  모델 생성 후 데이터 학습
from sklearn.linear_model import LogisticRegression #로지스틱 회귀 알고리즘 사용을 위해 import
# 로지스틱 회귀 모델의 penalty 속성으로 제약 방식 지정해서 모델을 만들 수 있다.
# penaly 속성의 기본값은 'l2'이고, L2 제약식을 적용한다. 'l1'은 L1 제약식을, 
# 'elasticnet'은 L1,L2 제약식을 모두 적용하고, 'None'은 제약방식을 지정하지 않는다.
clf = LogisticRegression(penalty='l2')
# 표준화된 학습 데이터 x_train와 학습 데이터에 따른 레이블 데이터 y_train으로 로지스틱 회귀 모델에 학습시킨다.
clf.fit(x_train, y_train)

# 로지스틱 회귀 분석 계수(가중치)와 상수항 (바이어스) 확인
print(clf.coef_) # 가중치
print(clf.intercept_) # 바이어스

(569, 30) (569,)
(426, 30) (143, 30) (426,) (143,)
[[-0.48143978 -0.41014209 -0.46725987 -0.51392775 -0.09769918  0.3020924
  -0.72884664 -0.7890253  -0.01424461  0.39299676 -1.08415325 -0.04728618
  -0.5669909  -0.74499028 -0.22229903  0.90896096 -0.05386185 -0.49962018
   0.17826258  0.71184481 -1.05809159 -0.95888641 -0.91097127 -0.97646535
  -0.41365059 -0.01729729 -0.82465772 -0.93295021 -0.76852329 -0.61983664]]
[0.36341572]


***
학습된 모델로 테스트 데이터를 예측한다.
***

In [3]:
#predict() 메소드의 인수로 표준화된 테스트 데이터를 넘겨서 예측한다.
predict = clf.predict(x_test)
print(predict)

[1 0 1 0 0 0 0 0 1 1 1 0 0 1 1 1 1 1 1 0 1 1 0 1 0 1 1 0 0 0 0 1 0 0 1 1 0
 1 0 1 1 1 1 1 1 0 1 1 1 0 0 0 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 0
 1 0 0 1 1 0 1 0 1 0 1 1 0 1 0 1 1 0 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0
 1 1 1 0 0 1 1 1 1 1 0 0 1 1 0 0 1 0 0 1 1 1 0 1 0 0 1 1 0 0 0 1]


In [4]:
# predict_proba() 메소드는 클래스의 속할 확률로 예측한다.
predict_proba = clf.predict_proba(x_test)
print(predict_proba)

# 종양은 2가지 (양성, 악성) 클래스로 예측되므로 두 개의 열로 결과가 이루어진 것을 알 수 있다.


[[1.49488533e-01 8.50511467e-01]
 [9.89656817e-01 1.03431830e-02]
 [9.87187552e-03 9.90128124e-01]
 [9.94890005e-01 5.10999525e-03]
 [8.94370796e-01 1.05629204e-01]
 [9.94653457e-01 5.34654285e-03]
 [9.99934190e-01 6.58097367e-05]
 [8.39850104e-01 1.60149896e-01]
 [6.71620121e-03 9.93283799e-01]
 [2.48219387e-02 9.75178061e-01]
 [1.16133334e-03 9.98838667e-01]
 [9.97966542e-01 2.03345776e-03]
 [9.81618379e-01 1.83816207e-02]
 [1.87756245e-03 9.98122438e-01]
 [3.41727616e-01 6.58272384e-01]
 [3.22621426e-02 9.67737857e-01]
 [5.94546271e-04 9.99405454e-01]
 [8.11730977e-03 9.91882690e-01]
 [6.99424444e-04 9.99300576e-01]
 [9.99931832e-01 6.81684178e-05]
 [2.52926669e-03 9.97470733e-01]
 [4.58182448e-03 9.95418176e-01]
 [9.99946728e-01 5.32715029e-05]
 [1.28963362e-02 9.87103664e-01]
 [9.92283615e-01 7.71638528e-03]
 [3.06703381e-02 9.69329662e-01]
 [2.40952072e-02 9.75904793e-01]
 [9.99991486e-01 8.51390903e-06]
 [1.00000000e+00 2.62520146e-13]
 [9.85741614e-01 1.42583858e-02]
 [9.999999

***
학습된 모델을 평가한다.
***

In [10]:
# 정확도 평가
# accuracy_score() 메소드의 인수를 테스트 데이터의 실제값, 예측값 순서로 넘겨서 정확도를 계산한다.
accuracy = accuracy_score(y_test, predict)
print(accuracy)

0.9790209790209791


In [11]:
# 정밀도 평가
precision = precision_score(y_test, predict, average=None)
print(precision)

[0.98148148 0.97752809]


In [12]:
# 재현률 평가
recall = recall_score(y_test, predict, average=None)
print(recall)

[0.96363636 0.98863636]


In [13]:
# f1_score 평가
f1Score = f1_score(y_test, predict, average=None)
print(f1Score)

[0.97247706 0.98305085]


In [14]:
# 혼동 행렬
conf_matrix = confusion_matrix(y_test, predict)
print(conf_matrix)

[[53  2]
 [ 1 87]]


In [16]:
#분류 리포트
class_report = classification_report(y_test, predict, target_names=['악성', '양성'])
print(class_report)

              precision    recall  f1-score   support

          악성       0.98      0.96      0.97        55
          양성       0.98      0.99      0.98        88

    accuracy                           0.98       143
   macro avg       0.98      0.98      0.98       143
weighted avg       0.98      0.98      0.98       143

